# CHAPTER 6: _Individual Fairness_

## The Preparations

### Installing the Libraries

In [ ]:
%%capture
!pip install solas-ai

### Loading the Libraries

In [ ]:
import pandas as pd
import numpy as np
import os

import statsmodels.api as sm
import solas_disparity as sd

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

RANDOM_SEED = 16180
np.random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

import xgboost as xgb
print(f"XGBoost Version: {xgb.__version__}")

XGBoost Version: 2.0.3


## Importing and Preparing the Data

In [ ]:
####
from google.colab import drive
drive.mount('/content/drive')

project_dir = 'drive/MyDrive/Responsible AI Book/data/processed/'
filename = 'simulated_df.csv.gz'
url = project_dir + filename
####

df = pd.read_csv(
    filepath_or_buffer= url,
    index_col=['train', 0]
)

Mounted at /content/drive


## Data Preperation and Model Building

In [ ]:
features = [
    'x1', 'x2', 'x3', 'x4', 'x5',
    'x6', 'x7', 'x8', 'x9', 'x10',
    'x11', 'x12', 'x13_B', 'x13_C',
    'x13_D', 'x13_E', 'x14_B',
    'x14_C', 'x14_D', 'x14_E'
]

df = df[['y_binary', 'minority', 'majority'] + features]
df['Total'] = 1
train = df.loc['train', :].copy()
valid = df.loc['valid', :].copy()
show_features = [
    'y_binary', 'minority', 'x1', 'x2',
    'x14_B', 'x14_C', 'x14_D', 'x14_E'
]
train[show_features].head()

,y_binary,minority,x1,x2,x14_B,x14_C,x14_D,x14_E
id,,,,,,,,
0,0,0,1.548523,1.143088,0,1,0,0
1,0,1,-0.710914,-0.552455,0,1,0,0
2,1,1,-0.015852,-0.153528,1,0,0,0
4,0,1,0.573155,0.382996,0,0,1,0
5,0,0,0.141408,-0.939697,0,0,1,0


In [ ]:
params= dict(
    n_estimators=750,
    max_depth=5,
    learning_rate=0.15,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_SEED,
    eval_metric='auc',
    early_stopping_rounds=10,
    tree_method='hist'
    #,device = 'cuda'  # Uncoment if you have access to a GPU
)

eval_set = [
    (train[features], train['y_binary']),
    (valid[features], valid['y_binary'])
]

xgb_model = xgb.XGBClassifier(**params).fit(
    X=train[features],
    y=train['y_binary'],
    eval_set=eval_set,
    verbose=100,
)
print(f"Number of Trees Used for Best Model: {xgb_model.best_iteration}")
print(f"Model AUC: {xgb_model.best_score:0.3f}")

[0]	validation_0-auc:0.73182	validation_1-auc:0.72809
[100]	validation_0-auc:0.83947	validation_1-auc:0.82777
[200]	validation_0-auc:0.85788	validation_1-auc:0.83860
[240]	validation_0-auc:0.86225	validation_1-auc:0.83972
Number of Trees Used for Best Model: 230
Model AUC: 0.840


In [ ]:
train['prediction'] = xgb_model.predict_proba(train[features])[:, 1]
valid['prediction'] = xgb_model.predict_proba(valid[features])[:, 1]

xgb_cutoff = train['prediction'].quantile(0.9)

train['Baseline Offer'] = np.where(
    train['prediction'] > xgb_cutoff, 1, 0
)
valid['Baseline Offer'] = np.where(
    valid['prediction'] > xgb_cutoff, 1, 0
)

print(
    f"Percent Favorable Outcomes for the Training and Validation Sets: "
    f"{train['Baseline Offer'].mean():0.2%}, {valid['Baseline Offer'].mean():0.2%}"
)

Percent Favorable Outcomes for the Training and Validation Sets: 10.00%, 9.85%


## Measuring Differential Performance

In [ ]:
relative_auc = sd.custom_disparity_metric(
    group_data=valid,
    protected_groups=['minority'],
    reference_groups=['majority'],
    group_categories=['Race'],
    label=valid['y_binary'],
    outcome=valid['prediction'],
    metric=roc_auc_score,
    ratio_threshold=lambda x: x < 0.90,
    difference_threshold=lambda x: x > 0.01,
)
keep_columns = [
    'Reference Group', 'ROC AUC SCORE',
    'Ratio', 'Difference', 'Practically Significant'
]
relative_auc.summary_table[keep_columns]

,Reference Group,ROC AUC SCORE,Ratio,Difference,Practically Significant
Group,,,,,
minority,majority,0.771921,0.846581,0.139889,Yes
majority,,0.911810,NaN,NaN,


## Measuring Differential Prediction

In [ ]:
rsmd = sd.residual_standardized_mean_difference(
    group_data=valid,
    protected_groups=['minority'],
    reference_groups=['majority'],
    group_categories=['Race'],
    label=valid['y_binary'],
    prediction=valid['prediction'],
    residual_smd_threshold=20,
    lower_score_favorable=True,  # DOUBLE CHECK THIS NOW!
)
keep_columns = ['Average Prediction', 'Average Label', 'Average Residual', 'Difference in Average Residual', 'Residual SMD', 'Practically Significant']

In [ ]:
print('rsmd for XGBoost model')
sd.ui.show(rsmd.summary_table[keep_columns])

rsmd for XGBoost model


Group,Average Prediction,Average Label,Average Residual,Difference in Average Residual,Residual SMD,Practically Significant
minority,0.197775,0.23,0.037035,0.073769,21.300234,Yes
majority,0.288754,0.25,-0.036734,,,


In [ ]:
rsmd.summary_table

,Reference Group,Group Category,Observations,Percent Missing,Total,Average Prediction,Average Label,Average Residual,Difference in Average Residual,Std. Dev. of Residuals,Residual SMD,P-Values,Practically Significant
Group,,,,,,,,,,,,,
minority,majority,Race,59915,0.0,29969,0.197775,0.234809,0.037035,0.073769,0.346328,21.300234,1.186298e-150,Yes
majority,,Race,59915,0.0,29946,0.288754,0.252020,-0.036734,NaN,0.346328,NaN,NaN,


## Identifying Individuals Harmed by a Model



### Using Interpretable Models with Accepted Features to Determine Individuals Harmed by a Model


In [ ]:
accepted_features = ['x1', 'x4','x9', 'x12']

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False, order='C')

poly_train = poly.fit_transform(train[accepted_features])
poly_train = pd.DataFrame(poly_train, columns=poly.get_feature_names_out(accepted_features), index=train.index)

poly_valid = poly.fit_transform(valid[accepted_features])
poly_valid = pd.DataFrame(poly_valid, columns=poly.get_feature_names_out(accepted_features), index=valid.index)
poly_valid[[x for x in poly_valid if x.find('x12') > -1]].head()

,x12,x1 x12,x4 x12,x9 x12,x12^2
id,,,,,
3,-0.279702,-0.048040,-0.484155,0.060801,0.078233
7,0.144728,0.139404,-0.180161,-0.149255,0.020946
9,-0.513789,-0.187635,0.123727,-0.042134,0.263979
27,0.737318,-1.869409,-0.682899,-0.273113,0.543638
29,-1.152369,0.855143,0.010456,0.206442,1.327954


In [ ]:
fairness_logit = sm.Logit(
    endog=train['y_binary'],
    exog=sm.add_constant(poly_train)
).fit()
fairness_logit.summary()

Optimization terminated successfully.
         Current function value: 0.494914
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               y_binary   No. Observations:               240085
Model:                          Logit   Df Residuals:                   240070
Method:                           MLE   Df Model:                           14
Date:                Fri, 12 Apr 2024   Pseudo R-squ.:                  0.1061
Time:                        01:26:30   Log-Likelihood:            -1.1882e+05
converged:                       True   LL-Null:                   -1.3292e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4251      0.009   -159.731      0.000      -1.443      -1.408
x1             0.0840      0.006     15.128      0.000       0.073       0.095
x4            -0.5177      0.006    -89.276      0.000      -0.529      -0.506
x9            -0.5246      0.006    -89.813      0.000      -0.536      -0.513
x12           -0.0032      0.005     -0.601      0.548      -0.014       0.007
x1^2           0.0174      0.004      4.556      0.000       0.010       0.025
x1 x4         -0.0199      0.006     -3.512      0.000      -0.031      -0.009
x1 x9         -0.0220      0.006     -3.862      0.000      -0.033      -0.011
x1 x12         0.0007      0.005      0.140      0.888      -0.009       0.011
x4^2           0.0362      0.004      8.772      0.000       0.028       0.044
x4 x9          0.0693      0.006     11.840      0.000       0.058       0.081
x4 x12         0.0076      0.005      1.421      0.155      -0.003       0.018
x9^2           0.0285      0.004      6.872      0.000       0.020       0.037
x9 x12        -0.0067      0.005     -1.248      0.212      -0.017       0.004
x12^2          0.0343      0.003      9.821      0.000       0.027       0.041
==============================================================================
"""

In [ ]:
train['logit_prediction'] = fairness_logit.predict(
    sm.add_constant(poly_train)
)

valid['logit_prediction'] = fairness_logit.predict(
    sm.add_constant(poly_valid)
)

logit_auc = metrics.roc_auc_score(
    y_true=valid['y_binary'],
    y_score=valid['logit_prediction']
)
xgb_auc = metrics.roc_auc_score(
    y_true=valid['y_binary'],
    y_score=valid['prediction']
)

print(f"Fair Logit ROC-AUC:       {logit_auc:0.2f}")
print(f"Original XGBoost ROC-AUC: {xgb_auc:0.2f}")

Fair Logit ROC-AUC:       0.72
Original XGBoost ROC-AUC: 0.84


In [ ]:
logit_residual_smd = sd.residual_standardized_mean_difference(
    group_data=valid,
    protected_groups=['minority'],
    reference_groups=['majority'],
    group_categories=['Race'],
    label=valid['y_binary'],
    prediction=valid['logit_prediction'],
    residual_smd_threshold=20,
    lower_score_favorable=True,
)

keep_columns = ['Average Prediction', 'Average Label', 'Average Residual', 'Difference in Average Residual', 'Residual SMD', 'Practically Significant']

In [ ]:
print('rsmd for logit model')
sd.ui.show(logit_residual_smd.summary_table[keep_columns])

rsmd for logit model


Group,Average Prediction,Average Label,Average Residual,Difference in Average Residual,Residual SMD,Practically Significant
minority,0.217501,0.23,0.017308,0.034507,8.571704,No
majority,0.269219,0.25,-0.017199,,,


In [ ]:
logit_residual_smd.summary_table

,Reference Group,Group Category,Observations,Percent Missing,Total,Average Prediction,Average Label,Average Residual,Difference in Average Residual,Std. Dev. of Residuals,Residual SMD,P-Values,Practically Significant
Group,,,,,,,,,,,,,
minority,majority,Race,59915,0.0,29969,0.217501,0.234809,0.017308,0.034507,0.402567,8.571704,9.069087e-26,No
majority,,Race,59915,0.0,29946,0.269219,0.252020,-0.017199,NaN,0.402567,NaN,NaN,


In [ ]:
xgb_cutoff = train['prediction'].quantile(0.9)
fair_logit_cutoff = train['logit_prediction'].quantile(0.9)

valid['XGBoost Offer'] = np.where(valid['prediction'] > xgb_cutoff, 'YES', 'NO')
valid['Logit Fair Offer'] = np.where(valid['logit_prediction'] > fair_logit_cutoff, 'Yes', 'NO')

In [ ]:
pd.crosstab(
    valid.loc[valid['minority'] == 1, 'Logit Fair Offer'],
    valid.loc[valid['minority'] == 1, 'XGBoost Offer'],
)

XGBoost Offer,NO,YES
Logit Fair Offer,,
NO,26828,1012
Yes,1343,786
